<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-11/blob/main/Primer%20modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer modelo

In [7]:
!wget https://unirioja-my.sharepoint.com/:x:/g/personal/joheras_unirioja_es/ETEcAGbnWDlHjhKzzxBFyAoBgCaD0GJZi-cX4-YIXtHBsA?e=LUSpNv -O datos.csv

--2021-05-18 11:55:39--  https://unirioja-my.sharepoint.com/:x:/g/personal/joheras_unirioja_es/ETEcAGbnWDlHjhKzzxBFyAoBgCaD0GJZi-cX4-YIXtHBsA?e=LUSpNv
Resolving unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)... 13.107.136.9
Connecting to unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unirioja-my.sharepoint.com/personal/joheras_unirioja_es/_layouts/15/Doc.aspx?sourcedoc=%7B66001c31-58e7-4739-8e12-b3cf1045c80a%7D&action=default&slrid=1b18c99f-30a2-b000-f874-6ae3d67a6d8b&originalPath=aHR0cHM6Ly91bmlyaW9qYS1teS5zaGFyZXBvaW50LmNvbS86eDovZy9wZXJzb25hbC9qb2hlcmFzX3VuaXJpb2phX2VzL0VURWNBR2JuV0RsSGpoS3p6eEJGeUFvQmdDYUQwR0paaS1jWDQtWUlYdEhCc0E_cnRpbWU9QjBDZDJfTVoyVWc&cid=d0e4f041-7996-4568-a8ea-5f58327930f7 [following]
--2021-05-18 11:55:40--  https://unirioja-my.sharepoint.com/personal/joheras_unirioja_es/_layouts/15/Doc.aspx?sourcedoc=%7B66001c31-58e7-4739-8e12-b3cf1045c80a%7D&